In [6]:


# Load datasets
gdp_data = pd.read_csv(r"D:\Finsight-Bharat\GDP_and_Major_Industrial_Sectors_of_Economy_Dataset.csv")
inflation_data = pd.read_csv(r"D:\Finsight-Bharat\CPI_Inflation_India_1970_2024.csv")
trade_data = pd.read_csv(r"D:\Finsight-Bharat\Foreign_Exchange_Trade_Balance_India_1980_2024.csv")
budget_data = pd.read_csv(r"D:\Finsight-Bharat\Budget_Allocation_India_1990_2024.csv")
nifty_data = pd.read_csv(r"D:\Finsight-Bharat\NIFTY 50-20-03-2024-to-20-03-2025.csv")

# Display first few rows of each dataset
print("GDP Data:\n", gdp_data.head())
print("\nInflation Data:\n", inflation_data.head())
print("\nTrade Balance Data:\n", trade_data.head())
print("\nBudget Allocation Data:\n", budget_data.head())
print("\nGDP Growth Rate Data:\n", nifty_data.head())


GDP Data:
   Financial Year Gross Domestic Product (in Rs. Cr) at 2004-05 Prices  \
0        1951-52                                           286,147      
1        1952-53                                           294,267      
2        1953-54                                           312,177      
3        1954-55                                           325,431      
4        1955-56                                           333,766      

  Agriculture & Allied Services (in Rs. Cr.) at 2004-05 Prices  \
0                                           147,216              
1                                           151,859              
2                                           163,553              
3                                           168,361              
4                                           166,906              

  Agriculture (in Rs. Cr.) at 2004-05 Prices  \
0                                   118,877    
1                                   123,822    
2        

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

In [2]:
def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)
    df["Financial Year"] = df["Financial Year"].str.split("-").str[0].astype(int)
    df.iloc[:, 1:] = df.iloc[:, 1:].replace(",", "", regex=True).astype(float)
    df.fillna(method="ffill", inplace=True)
    return df

In [3]:
def predict_gdp(models, start_year, end_year):
    future_years = np.array(range(start_year, end_year + 1)).reshape(-1, 1)
    predictions = {sector: models[sector].predict(future_years) for sector in models.keys()}
    percentage_increase = {}
    for sector, values in predictions.items():
        initial = values[0]
        final = values[-1]
        increase = ((final - initial) / initial) * 100 if initial != 0 else 0
        percentage_increase[sector] = increase
    
    report_data = {"Year": future_years.flatten()}
    for sector, values in predictions.items():
        report_data[sector] = values.flatten()
    report_df = pd.DataFrame(report_data)
    report_df["Percentage Increase"] = report_df.iloc[:, 1:].pct_change().sum(axis=1) * 100
    
    file_name = f"GDP_Prediction_Report_{start_year}_to_{end_year}.csv"
    report_df.to_csv(file_name, index=False)
    print(f"Report saved as {file_name}")
    
    print("\nPredicted GDP Contribution by Sector:")
    print(report_df)
    print("\nPercentage Increase from Start to End Year:")
    for sector, increase in percentage_increase.items():
        print(f"  {sector}: {increase:.2f}%")
    
    return future_years.flatten(), predictions, percentage_increase, file_name


In [4]:
def visualize_predictions(df, future_years, predictions):
    plt.figure(figsize=(12, 6))
    for sector, values in predictions.items():
        plt.plot(df["Financial Year"], df[sector], label=f"Historical {sector}", linestyle="dotted")
        plt.plot(future_years, values, label=f"Predicted {sector}", linestyle="solid")
    plt.xlabel("Year")
    plt.ylabel("GDP Contribution (in Rs. Cr.)")
    plt.title("Sector-wise GDP Contribution Prediction")
    plt.legend()
    plt.grid(True)
    plt.show()

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib import colors
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Spacer
from datetime import datetime

# Load GDP Data
def load_gdp_data(file_path):
    df = pd.read_csv(file_path)
    df.dropna(inplace=True)
    return df

# Train Polynomial Regression Model
def train_gdp_model(df):
    models = {}
    for sector in df.columns[1:]:
        X = df[['Year']].values
        y = df[sector].values
        models[sector] = np.poly1d(np.polyfit(X.flatten(), y, 2))  # Polynomial Regression
    return models

# Predict GDP
def predict_gdp(models, start_year, end_year):
    future_years = np.arange(start_year, end_year + 1)
    predictions = {sector: models[sector](future_years) for sector in models}
    return future_years, predictions

# **Enhanced Visualization**
def visualize_predictions(df, future_years, predictions, save_path="gdp_plot.png"):
    plt.figure(figsize=(12, 6))
    colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'orange', 'purple']
    linestyles = ['-', '--', '-.', ':']

    for i, (sector, values) in enumerate(predictions.items()):
        plt.plot(
            future_years, 
            values, 
            linestyle=linestyles[i % len(linestyles)], 
            marker='o', 
            markersize=5, 
            color=colors[i % len(colors)], 
            label=sector
        )

    plt.xlabel("Year", fontsize=14, fontweight="bold")
    plt.ylabel("GDP Contribution", fontsize=14, fontweight="bold")
    plt.title(" GDP Sector-wise Forecast", fontsize=16, fontweight="bold", color="darkblue")
    
    # Improved legend placement (avoiding overlap)
    plt.legend(loc="upper left", bbox_to_anchor=(1.02, 1), borderaxespad=0)  
    
    plt.grid(alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout(rect=[0, 0, 0.8, 1])  # Adjust layout to prevent overlap
    plt.savefig(save_path, dpi=300)
    plt.close()
    
    return save_path

# ** New: Stylish PDF Report with Table First, Graph Below**
def generate_pdf_report(future_years, predictions, visualization_path, start_year, end_year):
    report_path = f"GDP_Report_{start_year}_to_{end_year}.pdf"
    doc = SimpleDocTemplate(report_path, pagesize=letter)
    
    elements = []

    # **Title Styling**
    from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
    from reportlab.platypus import Paragraph
    
    styles = getSampleStyleSheet()
    title_style = ParagraphStyle(
        "TitleStyle",
        parent=styles["Title"],
        fontSize=18,
        textColor=colors.darkblue,
        spaceAfter=10,
        alignment=1  # Center Align
    )
    
    elements.append(Paragraph(f"GDP Report ({start_year}-{end_year})", title_style))
    elements.append(Spacer(1, 12))

    # **Create Table with Word Wrapping**
    from reportlab.lib.units import inch
    
    # **Fix: Adjust column widths dynamically**
    col_widths = [1.2 * inch] + [1.5 * inch] * len(predictions.keys())  

    data = [["Year"] + [f"{sector[:10]}..." if len(sector) > 12 else sector for sector in predictions.keys()]]  # Shorten long sector names
    
    for i, year in enumerate(future_years):
        row = [year] + [f"{predictions[sector][i]:.2f}" for sector in predictions]
        data.append(row)

    table = Table(data, colWidths=col_widths)

    # **Enhanced Table Styling**
    style = TableStyle([
        ("BACKGROUND", (0, 0), (-1, 0), colors.darkblue),
        ("TEXTCOLOR", (0, 0), (-1, 0), colors.whitesmoke),
        ("ALIGN", (0, 0), (-1, -1), "CENTER"),
        ("FONTNAME", (0, 0), (-1, 0), "Helvetica-Bold"),
        ("FONTNAME", (0, 1), (-1, -1), "Helvetica"),
        ("FONTSIZE", (0, 1), (-1, -1), 9),  # Fix: Reduce font size slightly to avoid overlap
        ("BOTTOMPADDING", (0, 0), (-1, 0), 8),
        ("BACKGROUND", (0, 1), (-1, -1), colors.lightgrey),
        ("GRID", (0, 0), (-1, -1), 1, colors.black),
    ])
    table.setStyle(style)
    elements.append(table)
    
    elements.append(Spacer(1, 20))  # Space before graph

    # **Insert Visualization Below**
    from reportlab.platypus import Image
    img = Image(visualization_path, width=400, height=300)
    elements.append(img)

    # **Build PDF**
    doc.build(elements)
    print(f"PDF report saved: {report_path}")
    return report_path


# ** Example Usage**
file_path = r"D:\Finsight-Bharat\GDP_Prediction_Report_2012_to_2020.csv"
df = load_gdp_data(file_path)
models = train_gdp_model(df)

# **User-defined range**
start_year = int(input("Enter start year: "))
end_year = int(input("Enter end year: "))

future_years, predictions = predict_gdp(models, start_year, end_year)
visualization_path = visualize_predictions(df, future_years, predictions)
pdf_report = generate_pdf_report(future_years, predictions, visualization_path, start_year, end_year)

print(" Visualization saved at:", visualization_path)
print(" Download report:", pdf_report)


PDF report saved: GDP_Report_2010_to_2015.pdf
 Visualization saved at: gdp_plot.png
 Download report: GDP_Report_2010_to_2015.pdf
